# Создание отчетов из отзывов (часть 1)
В этой серии ноутбуков мы покажем, как из большого количества неструктурированных отзывов пользователей можно выделить тематические группы и проблемные области, а затем на основе полученных данных генерировать отчёты.

_Пример итогового отчёта: [report.pdf](media/report.pdf)_

Для анализа используются отзывы с [Kaggle датасета](https://www.kaggle.com/datasets/mikhaildvoshansky/bank-reviews)

**Структура серии:**
1. [Часть 1](part_1.ipynb) – кластеризация и визуальный анализ отзывов.  
2. [Часть 2](part_2.ipynb) – генерация отчёта на основе кластеров с использованием LLM.

**Инструменты:**
- [GigaChat](https://developers.sber.ru/docs/ru/gigachat/api/overview) – генерация эмбеддингов для текста.
- [Arize Phoenix](https://phoenix.arize.com/) – визуализация эмбеддингов и анализ кластеров.

---

## Введение

Исходными данными для анализа являются отзывы пользователей и их краткие суммаризации. 

Суммаризация помогает устранить лишний шум в тексте и улучшить качество последующей кластеризации.

В данном ноутбуке проводится предварительный анализ отзывов банка, который включает следующие этапы:
- Преобразование суммаризаций в эмбеддинги с помощью GigaChat;
- Центрирование эмбеддингов для улучшения визуализации;
- Визуализация эмбеддингов с использованием Arize Phoenix, настройка параметров кластеризации и оценка результатов.

---

## Подготовка к запуску

Перед выполнением ноутбука необходимо установить следующие библиотеки:

In [ ]:
%%capture --no-stderr
%pip install "arize-phoenix[embeddings]" "kagglehub[pandas-datasets]" langchain_gigachat pandas numpy python-dotenv

---

## Инициализация подключения GigaChat

In [10]:
import getpass
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

if "GIGACHAT_CREDENTIALS" not in os.environ and "GIGACHAT_USER" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Credentials от GigaChat")
    
scope = "GIGACHAT_API_PERS" # Возможно также: GIGACHAT_API_CORP / GIGACHAT_API_B2B

In [11]:
from langchain_gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    verify_ssl_certs=False,
    scope=scope,
    model="EmbeddingsGigaR"
)

---

## Загрузка и изучение данных

Загружаем DataFrame с отзывами пользователей. Здесь хранятся оригинальный текст, суммаризация, оценка и другая информация.

Также здесь хранятся проставленные кластера, к которым мы отнесли отзывы. Они используются во второй части.
В этой части, эта колонка нам не понадобится.

In [12]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "mikhaildvoshansky/bank-reviews",
  "bank_reviews.csv",
).drop(["cluster_id"], axis=1)
df

,Unnamed: 0,text,summary,title,grade,comment_count,is_countable,timestamp
0,0,<p>Добрый день. Хоум банк передал все права в ...,"Пользователь сообщает, что его кредит был пере...",Передали мою кредитную карту в другой банк,1.0,0,NaN,2025-02-06 12:20:50.269000+00:00
1,1,<p>27.11 на кредитном счете 6889 были заблокир...,Пользователь сообщил о блокировке 10 000 рубле...,Незаконное удержание средств !!!!!!!,1.0,0,NaN,2025-02-06 12:20:50.269000+00:00
2,2,<p>Оформляла кредитную карту Хоум банк в июне ...,Пользователь оформил кредитную карту Хоум банк...,Не дали сертификат озон,3.0,0,NaN,2025-02-06 12:20:50.269000+00:00
3,3,"<p>Я пришел в офис, чтобы его закрыть досрочно...",Пользователь столкнулся с проблемой отсутствия...,Класс),5.0,0,0.0,2025-02-06 12:20:50.269000+00:00
4,4,<p>У меня есть кредит в Хоум банке.</p>\r\n<p>...,Пользователь столкнулся с проблемой при оплате...,Просрочка по вине банка,1.0,0,NaN,2025-02-06 12:20:50.269000+00:00
...,...,...,...,...,...,...,...,...
1015,1015,"<p>Захотел подать заявку на кредитную карту ""1...",**Краткий пересказ сообщения:**\n\nПользовател...,"Что надо знать перед оформлением кредитки ""120...",5.0,2,1.0,2025-02-06 12:20:50.269000+00:00
1016,1016,<p>Брала рассрочку в Хоум банке. Каждый месяц ...,Пользователь сообщает о проблемах с оплатой по...,Хоум Банк не дает совершить последний платеж,1.0,0,1.0,2025-02-06 12:20:50.269000+00:00
1017,1017,"<p>Здравствуйте, хочу поблагодарить банк Хоум ...",**Краткий пересказ:**\nПользователь благодарит...,Увеличили лимит на денежный перевод,5.0,0,0.0,2025-02-06 12:20:50.269000+00:00
1018,1018,<p>Здравствуйте! Я очень довольна что в этом б...,Пользователь выражает удовлетворение качеством...,Помогут даже в выходные дни,5.0,0,1.0,2025-02-06 12:20:50.269000+00:00


---

## Преобразование суммаризаций в эмбеддинги

In [13]:
import numpy as np
summary_emb = list(map(np.array, embeddings.embed_documents(list(df['summary']))))

### Центрирование эмбеддингов

In [14]:
df['embeddings'] = summary_emb
document_vectors_centroid = df["embeddings"].mean(axis=0)
df["centered_embeddings"] = df["embeddings"].apply(
    lambda x: x - document_vectors_centroid
)

---

## Визуализация и кластеризация с Arize Phoenix

Настроим схему для Arize Phoenix и запустим приложение для визуального анализа эмбеддингов.

### Определение схемы данных для Phoenix

Указываем, какие столбцы являются эмбеддингами и дополнительные фичи, которые можно использовать при анализе.

Например, мы сможем найти кластера с самыми низкими оценками

In [16]:
import phoenix

document_vectors_schema = phoenix.Schema(
    prompt_column_names=phoenix.EmbeddingColumnNames(
        raw_data_column_name="summary",
        vector_column_name="centered_embeddings",
    ),
    feature_column_names=[
        'summary',
        'text',
        'title',
        'grade',
        'comment_count',
        'is_countable'
    ]
)

document_vectors_ds = phoenix.Inferences(
    dataframe=df,
    schema=document_vectors_schema,
    name="document_vectors",
)

### Запуск Arize Phoenix

При запуске приложения в консоли будет выведен адрес, по которому можно открыть Phoenix в браузере.  
Также обратите внимание, что если запущена другая сессия, предыдущая будет остановлена.

In [17]:
session = phoenix.launch_app(document_vectors_ds)

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


_После запуска перейдите по адресу, указанному в выводе (например, [http://localhost:6006/](http://localhost:6006/))._

### Инструкции по работе с Phoenix

Зайдя по ссылке в интерфейс Arize мы увидим первый экран с информацией о схеме. Выберем `centered_embeddings`
<img src="media/arize/screen_1.png" width="800">

Далее мы увидим, как наши эмбединги с 2560 размерностью преобразовались в 3-ех мерную размерность.

Сейчас показывается 500 обращений, чтобы просмотреть все, нам нужно проставить значение `n_samples` — `2000`.
<img src="media/arize/screen_2.png" width="800">

Также мы можем настроить следующие параметры

Для кластеризации [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/index.html):
* **min_cluster_size** — минимальное количество точек, чтобы группа считалась кластером. Если кластер меньше этого значения, он будет отнесён к шуму. Чем больше значение, тем крупнее и устойчивее должны быть кластеры.
* **cluster_min_samples** — определяет, насколько плотными должны быть кластеры. Это похоже на min_samples в DBSCAN: более высокие значения делают алгоритм менее чувствительным к шуму, помогая находить только плотные кластеры.
* **cluster_selection_epsilon** — ограничивает максимальное расстояние между точками внутри кластера. Полезно, если нужно предотвратить образование слишком разреженных кластеров.

Для снижения размерности [UMAP](https://umap-learn.readthedocs.io/en/latest/):
* **n_neighbors** — определяет, сколько ближайших точек учитывается при построении структуры. Малые значения помогают выявить локальные паттерны, большие – сохранить глобальную картину.
* **min_distance** — регулирует, насколько плотно расположены точки после сжатия. Малые значения позволяют точкам слипаться в кластеры, большие – раздвигают их.

Я использовал стандартные параметры, только поменял `min_cluster_size` на `15`

Давайте теперь выставим метрику `grade` для наших кластеров 

<img src="media/arize/screen_3.png" width="800">

Теперь мы видим среднюю оценку из отзывов в наших кластерах.

Посмотрим, про что пишут в самом крупном

<img src="media/arize/screen_4.png" width="800">

Ага, тут в основном обращения, с жалобами на проблемы с кэшбэком.

Мы можем дальше поизучать кластера — понять какие темы поднимаются в отзывах. Из этого выявить проблемные области и категории.
 
Давайте экспортируем кластера

Нажимаем кнопку Export

<img src="media/arize/screen_5.png" width="800">


Смотрим на экспортируемый датасет и сохраняем его

In [18]:
# Просмотр экспортированного датасета
phoenix.active_session().exports[0]

,text,summary,title,grade,comment_count,is_countable,centered_embeddings,prediction_id,timestamp,__phoenix_cluster_id__
0,<p>Добрый день. Хоум банк передал все права в ...,"Пользователь сообщает, что его кредит был пере...",Передали мою кредитную карту в другой банк,1.0,0,NaN,"[1.4701072244083182, -0.16595498206568704, 0.2...",f675aeb6-3604-4513-8a92-45a8e29dd3e7,2025-02-18 11:41:15.144000+00:00,16
1,<p>27.11 на кредитном счете 6889 были заблокир...,Пользователь сообщил о блокировке 10 000 рубле...,Незаконное удержание средств !!!!!!!,1.0,0,NaN,"[1.0262595681583182, 1.6248165023093128, -0.78...",1bea55ef-0adf-4485-b716-c88d18ec8303,2025-02-18 11:41:15.144000+00:00,20
2,<p>Оформляла кредитную карту Хоум банк в июне ...,Пользователь оформил кредитную карту Хоум банк...,Не дали сертификат озон,3.0,0,NaN,"[-1.4053810568416818, -0.720398341440687, 0.97...",bbff65e4-4a74-468d-973d-7f5d4f1aa167,2025-02-18 11:41:15.144000+00:00,0
3,"<p>Я пришел в офис, чтобы его закрыть досрочно...",Пользователь столкнулся с проблемой отсутствия...,Класс),5.0,0,0.0,"[0.0750876931583182, -1.6886600601906872, -0.7...",1b39a49a-b4b1-4f38-8e96-17e18fb25438,2025-02-18 11:41:15.144000+00:00,3
4,<p>У меня есть кредит в Хоум банке.</p>\r\n<p>...,Пользователь столкнулся с проблемой при оплате...,Просрочка по вине банка,1.0,0,NaN,"[0.022353318158318203, 0.13214072105931296, -1...",4030f24f-31e7-4de0-ad59-ba6955fa9481,2025-02-18 11:41:15.144000+00:00,26
...,...,...,...,...,...,...,...,...,...,...
887,"<p>Оформил дебетовую виртуальную карту, заявле...",Пользователь оформил бесплатную виртуальную де...,Некачественное обслуживание,1.0,0,0.0,"[-1.9151466818416818, -0.08905068519068704, 0....",2e10ced7-9138-423e-9f3b-19024135608f,2025-02-18 11:41:15.144000+00:00,6
888,"<p>Захотел подать заявку на кредитную карту ""1...",**Краткий пересказ сообщения:**\n\nПользовател...,"Что надо знать перед оформлением кредитки ""120...",5.0,2,1.0,"[-0.6573341818416818, 0.41046103355931296, 1.9...",00e892d0-34be-4b2c-838e-a2b8c91e3a32,2025-02-18 11:41:15.144000+00:00,24
889,<p>Брала рассрочку в Хоум банке. Каждый месяц ...,Пользователь сообщает о проблемах с оплатой по...,Хоум Банк не дает совершить последний платеж,1.0,0,1.0,"[0.7323142556583182, -0.551514064096937, -1.23...",66040f8b-4bf3-46ce-b3fa-88f3a210f874,2025-02-18 11:41:15.144000+00:00,15
890,"<p>Здравствуйте, хочу поблагодарить банк Хоум ...",**Краткий пересказ:**\nПользователь благодарит...,Увеличили лимит на денежный перевод,5.0,0,0.0,"[0.008681443158318203, -0.38092080237818704, -...",1db93f61-1227-4442-9d01-9f0614bfbfbe,2025-02-18 11:41:15.144000+00:00,9


In [19]:
# Сохранение экспортированного датасета для дальнейшего использования
phoenix.active_session().exports[0].to_pickle('bank_clusterized.pkl')

-----
## Заключение

В данной части мы:
- Загрузили отзывы и преобразовали суммаризации в эмбеддинги и центрировали их.
- Настроили визуализацию и кластеризацию с помощью Arize Phoenix.

В [Части 2](part_2.ipynb) на основе экспортированных кластеров мы продолжим анализ и сгенерируем итоговый отчёт.